In [3]:
import os
import glob
import time
import math

import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.enums import Resampling

In [4]:
# Calculate Surface Area of Grid Cell
def calculate_surface_area(ds):
    # Find the lon size
    lon_interval_size = ds.variables['lon'][1] - ds.variables['lon'][0]
    # Find the lat size
    lat_interval_size = ds.variables['lat'][1] - ds.variables['lat'][0]
    surface_area_array = np.array([6371000*math.radians(lat_interval_size)*6371000*math.radians(lon_interval_size)*math.cos(math.radians(lat)) for lat in ds.lat.values])
    return surface_area_array

In [5]:
# Unit Factors Dict
unit_factors = {"mm": 1000, "kg/m2": 1000, "kg/m^2": 1000, "cm": 100, "dm": 10, "m": 1, "km": .001, "kg m-2": 1000}

In [6]:
# For a given gldas dataset and surface area data array calculate anomalies for each variable
def calculate_gldas_anomalies(ds, surface_area_da):
    for var in ds.variables:
        if var not in ['lat', 'lon', 'time', 'spatial_ref']:
            # Calculate variable mean from 2004 to 2009
            var_mean = ds.sel(time=slice('2004-01-01', '2009-12-31')).mean('time')[var]
#             ds.sel(time=slice('2004-01-01', '2009-12-31')).mean('time')[var]
            var_factor = unit_factors[ds[var].units]
            ds[var] = (ds[var] - var_mean)/var_factor * surface_area_da
            ds[var] = ds[var]*100/surface_area_da
    return ds

In [7]:
### import GLDAS data, filter to CRB, combine into one dataframe 

gldas_path = "/home/yhuang21/remoteData/GLDAS/NOAH_monthly_L4/"

gldas_df = pd.DataFrame()

#Iterating through files in path
for filename in os.listdir(gldas_path):
    
    if filename.endswith(".nc4"):
        
        #Reading in data as xarray then converting to DataFrame
        xd = xr.open_dataset(gldas_path+str(filename))
        xd_df = xd.to_dataframe()

        xd_df.reset_index(inplace=True)

        xd_df = xd_df[(xd_df.lon> lon_min) & (xd_df.lon < lon_max)]
        xd_df = xd_df[(xd_df.lat> lat_min) & (xd_df.lat < lat_max)]

        #Extracting only needed columns 
        df_slice = xd_df[["time", "lon", "lat", "SWE_inst", "RootMoist_inst"]]
        df_slice = df_slice.drop_duplicates()

        gldas_df = pd.concat([gldas_df, df_slice], axis=0)

gldas_df = gldas_df.rename(columns={'SWE_inst': 'SWE_inst_kg/m2', 'RootMoist_inst': 'RootMoist_inst_kg/m2'})

In [ ]:
# Compute surface area for pixel with dimension 0.25 x 0.25 degree.
EARTH_RADIUS_KM = 6371 
gldas_df['surface_area_km2_0.25'] = EARTH_RADIUS_KM * np.radians(.25) * EARTH_RADIUS_KM * np.radians(.25) * np.cos(np.radians(gldas_df['lat']))

In [ ]:
# GLDAS Grid Cell Area
areas = calculate_surface_area(gldas_df)

# surface_area_da = (xr.DataArray(areas, dims=['lat'],
#                                 coords={'lat': gldas_clsm_ds.lat})
#                    .expand_dims(lon=len(gldas_clsm_ds.coords['lon']))
#                    .assign_coords(lon=gldas_clsm_ds.lon)
#                    .transpose('lat', 'lon')
# #                    .drop('spatial_ref')
#                   )
# surface_area_da